<h5>This code is part of XLR. It implements exponential learning rate decay model training. Date: 01/09/2025</h5>

<h5>Contact: rakibul.haque@utsa.edu</h5>

<h5>Cite as: R. U. Haque and P. Markopoulos,"XLR: A Universal Framework for Learning rate Adaptation via Exponential Range Exploration", 2025</h5>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from collections import Counter
import random
import copy
import torch.nn.functional as F
import sys
import time
import pickle
import pandas as pd
import random
import queue
from collections import deque
from torch import Tensor
from typing import Type
import torchvision.models as models
import math
import matplotlib.pyplot as plt
from torch.utils.data import random_split, DataLoader
import math
import gc
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import math
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import _LRScheduler

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
initial_lr = 0.01#0.00001 #0.01
weight_decay = 0.0005
momentum = 0.9
batch_size = 128
num_epochs = 50
phase_boundaries = [80, 120]  # Epochs where LR would normally be reduced by 10x


# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
# test_dataset = torchvision.datasets.CIFAR10(
#     root='./data', train=False, download=True, transform=transform_test)


subset_dataset = torch.load('val_dataset.pth',weights_only=False)
remaining_dataset = torch.load('test_dataset.pth',weights_only=False)
val_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(remaining_dataset, batch_size=batch_size, shuffle=False)

class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu5 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)  # Change output to 10 for CIFAR-10 classes

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.relu5(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

model = cnn().to(device)
file_path = "s_cnn.pth"
initial_weights=torch.load(file_path, weights_only=True)
# Print("Model's initial weights", initial_weights)
model.load_state_dict(initial_weights)

# Custom LR scheduler implementing the exponential schedule
class TaperedExponentialLR(_LRScheduler):
    def __init__(self, optimizer, initial_lr, weight_decay, momentum, phase_boundaries, last_epoch=-1):
        self.initial_lr = initial_lr
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.phase_boundaries = phase_boundaries
        self.current_phase = 0
        self.alphas = []
        self.compute_alphas()
        super(TaperedExponentialLR, self).__init__(optimizer, last_epoch)

    def compute_alpha(self, lr):
        a = 1
        b = -(1 + self.momentum - self.weight_decay * lr)
        c = self.momentum
        discriminant = b**2 - 4*a*c
        return (-b + math.sqrt(discriminant)) / (2*a)

    def compute_alphas(self):
        self.alphas = []
        for i in range(len(self.phase_boundaries) + 1):
            phase_lr = self.initial_lr * (0.1 ** i)
            self.alphas.append(self.compute_alpha(phase_lr))

    def get_lr(self):
        epoch = self.last_epoch
        # Determine current phase
        self.current_phase = 0
        for i, boundary in enumerate(self.phase_boundaries):
            if epoch >= boundary:
                self.current_phase = i + 1
        
        # Compute base LR for phase
        if self.current_phase == 0:
            base_lr = self.initial_lr * (self.alphas[0] ** -1)
        else:
            prev_phase_lr = self.initial_lr * (0.1 ** (self.current_phase - 1))
            base_lr = self.initial_lr * (self.alphas[self.current_phase] ** -1) * \
                     (self.alphas[self.current_phase - 1] ** -1) * (0.1 ** self.current_phase)
        
        # Compute epochs in current phase
        prev_boundary = self.phase_boundaries[self.current_phase - 1] if self.current_phase > 0 else 0
        epochs_in_phase = epoch - prev_boundary
        
        current_lr = base_lr * (self.alphas[self.current_phase] ** (-2 * epochs_in_phase))
        return [current_lr for _ in self.base_lrs]

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=initial_lr, momentum=momentum, weight_decay=0)  # No weight decay!
scheduler = TaperedExponentialLR(optimizer, initial_lr, weight_decay, momentum, phase_boundaries)

# Training loop
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    print(f'Epoch {epoch+1}, LR: {scheduler.get_lr()[0]:.6f}')
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 99:
            print(f'[{epoch+1}, {batch_idx+1}] loss: {running_loss/100:.3f} | Train acc: {100.*correct/total:.1f}%')
            running_loss = 0.0
    
    scheduler.step()

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    acc = 100. * correct / total
    print(f'Test Epoch: {epoch+1}, Loss: {test_loss/(batch_idx+1):.4f}, Acc: {acc:.2f}%')
    return acc

best_acc = 0
for epoch in range(num_epochs):
    train(epoch)
    acc = test(epoch)
    
    # Save best model
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'best_model.pth')

print(f'Best test accuracy: {best_acc:.2f}%')

Files already downloaded and verified
Epoch 1, LR: 0.010001
[1, 100] loss: 2.295 | Train acc: 12.3%
[1, 200] loss: 2.104 | Train acc: 17.5%
[1, 300] loss: 1.995 | Train acc: 20.2%
Test Epoch: 1, Loss: 1.6998, Acc: 39.55%
Epoch 2, LR: 0.010002
[2, 100] loss: 1.787 | Train acc: 33.7%
[2, 200] loss: 1.702 | Train acc: 35.3%
[2, 300] loss: 1.644 | Train acc: 36.7%
Test Epoch: 2, Loss: 1.4240, Acc: 46.89%
Epoch 3, LR: 0.010003
[3, 100] loss: 1.560 | Train acc: 42.5%
[3, 200] loss: 1.527 | Train acc: 43.1%
[3, 300] loss: 1.518 | Train acc: 43.5%
Test Epoch: 3, Loss: 1.2835, Acc: 54.07%
Epoch 4, LR: 0.010004
[4, 100] loss: 1.421 | Train acc: 48.2%
[4, 200] loss: 1.412 | Train acc: 48.5%
[4, 300] loss: 1.392 | Train acc: 48.8%
Test Epoch: 4, Loss: 1.1638, Acc: 58.09%
Epoch 5, LR: 0.010005
[5, 100] loss: 1.313 | Train acc: 52.3%
[5, 200] loss: 1.286 | Train acc: 52.7%
[5, 300] loss: 1.289 | Train acc: 52.9%
Test Epoch: 5, Loss: 1.0708, Acc: 61.71%
Epoch 6, LR: 0.010006
[6, 100] loss: 1.229 | Tr